In [113]:
import os
import pandas as pd
import json
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import nltk
import re

from collections import defaultdict

import numpy as np


from langdetect import detect
from langdetect import DetectorFactory
import pickle

import spacy



In [114]:
current = os.getcwd() #current directory
data_folder = '/CORD-19-research-challenge/'

#The data folders
folders = ['document_parses']

#Counting the files
paths = dict()
for folder in folders:
    f = current + data_folder + folder + '/pdf_json/'
    filenames = os.listdir(f)
    paths[folder] = filenames

for key in paths.keys():
    print("Number of articles retrieved from {}:".format(key), len(paths[key]))

Number of articles retrieved from document_parses: 68905


In [115]:
#loading the data from the paths into a list of json text data as dict
def load(folder:str) -> list:
    all_files = []
    for filename in paths[folder]:
        file = os.path.join(current + data_folder + folder + '/pdf_json/' + filename)
        f = json.load(open(file,'rb'))
        all_files.append(f)
    return all_files

#loaded = ['biorxiv_medrxiv','comm_use_subset','custom_license','noncomm_use_subset','arxiv']

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_authors(authors):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        
        name_ls.append(name)
    
    return ", ".join(name_ls)

#Take the json, format all the data we need and place into dataframe
def finish(data):
    clean = []
    for f in data:
        features = [
            f['paper_id'],
            f['metadata']['title'],
            format_authors(f['metadata']['authors']),
            format_body(f['abstract']),
            format_body(f['body_text']),
        ]
        clean.append(features)

    column_names = [
        'paper_id',
        'title',
        'authors',
        'abstract',
        'body_text'
    ]

    df = pd.DataFrame(clean,columns=column_names)
    return df

In [116]:
# bio = load('biorxiv_medrxiv')
# print('loaded bio')
# comm_use_subset = load('comm_use_subset')
# print('loaded comm_use_subset')

# custom_license = load('custom_license')
# print('loaded custom_license')

# noncomm_use_subset = load('noncomm_use_subset')
# print('loaded noncomm_use_subset')

# arxiv = load('arxiv')
# print('loaded arxiv')

art = load('document_parses')
print('loaded documents')

loaded documents


In [117]:
# b = finish(bio)
# print('finshed bio')

# comm = finish(comm_use_subset)
# print('finshed comm_use_subset')

# custom = finish(custom_license)
# print('finshed custom_license')

# n = finish(noncomm_use_subset)
# print('finshed noncomm_use_subset')

# a = finish(arxiv)
# print('finished arxiv')

#Create combined dataset
big = finish(art)
print("finshed")

KeyboardInterrupt: 

In [ ]:
#Create lemmatized body text column in data
stop_words = set(stopwords.words('english'))
custom_stop_words = ['fig','virus','il-','ifn','virus','cell','cells','data','model','medrxiv','data','disease','licence',
                    'certified','version','author','nc','nd','cc','rsv','reuse','peer','review','international','by',
                    'preprint','respondent','http','https','copyright','patient','introduction']

def stop_word_add(custom,words:set):
    for word in custom:
        words.add(word)
    return words

lemmatizer = WordNetLemmatizer()
def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens = [word for word in doc.ents]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', str(token)) and len(str(token)) > 1 and str(token) not in stop_words:
            filtered_tokens.append(str(token))       
    stems = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    l = ', '.join(stems)
    return l



In [ ]:
stop_words = stop_word_add(custom_stop_words,stop_words)

nlp = spacy.load('en_core_sci_lg')
nlp.max_length = 5000000

big['body_text_new'] = big.apply(lambda row: tokenize_and_lemmatize(row['body_text'].lower()), axis=1)
big['abstract_new'] = big.apply(lambda row: tokenize_and_lemmatize(row['abstract'].lower()), axis=1)


big['title'] = big['title'].str.lower()

#Export to csv file
print('saving to csv')
big.to_csv('data.csv',index=False)

In [34]:
# USEmodel = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# train = USEmodel(data.body_text)
# train_m = tf.train.Checkpoint(v=tf.Variable(train))

# train_m.f = tf.function( lambda  x: exported_m.v * x, input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

# model = train_m.v.numpy()

# pickle.dump(model, open('full_tfidf_model.pk', 'wb'))

In [35]:
# count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
# count_data = count_vectorizer.fit_transform(data['body_text_new'])

In [36]:
# def SearchDocument(query,size=100):
#     q =[query]
#     Q_Train = USEmodel(q)
    
#     linear_similarities = linear_kernel(Q_Train, model).flatten() 
    
#     Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
#     #print(Top_index_doc)
#     linear_similarities.sort()
#     find = pd.DataFrame()
#     for i,index in enumerate(Top_index_doc):
#         find.loc[i,'index'] = str(index)
#         find.loc[i,'Paper_ID'] = data['paper_id'][index] 
#         find.loc[i,'Title'] = data['title'][index] 
#         find.loc[i,'abstract'] = data['abstract'][index]
#     for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
#         find.loc[j,'Score'] = simScore
        
#     if size==1:
#         if find.isnull().values.any():
#             print("Query: ",query,".  Title of the Research Paper is missing, Paper ID is:",find.loc[0,'Paper_ID'],"\n")
#         else:
#             print("Query: ",query,".  Title of the Research Paper:",find.loc[0,'Title'],"\n")
#     else:
#         return find